In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [2]:
# Define constants
DATASET_PATH = 'dataset/'  # Path to your dataset folder
IMAGE_SIZE = (128, 128)    # Resize images for training
BATCH_SIZE = 8            # Batch size for training
EPOCHS = 10                # Number of epochs for training

In [3]:
# Split dataset files into training and validation sets
all_files = [f for f in os.listdir(DATASET_PATH) if f.endswith('.npz')]
train_files, val_files = train_test_split(all_files, test_size=0.2, random_state=42)

In [4]:
# Data generator function (with memory safe handling and limiting images)
def data_generator(files, path, batch_size):
    while True:
        np.random.shuffle(files)  # Shuffle data at the start of each epoch
        for i in range(0, len(files), batch_size):
            batch_files = files[i:i + batch_size]
            batch_images = []
            for file in batch_files:
                try:
                    data = np.load(os.path.join(path, file))
                    color_images = data['colorImages']
                    color_images = color_images[:10]  # Limit to 10 images per file
                    for img in color_images:
                        # Resize and normalize images
                        img_resized = cv2.resize(img[:, :, :3], IMAGE_SIZE) / 255.0
                        batch_images.append(img_resized)
                except Exception as e:
                    print(f"Error loading file {file}: {e}")
                    
            batch_images = np.array(batch_images)
            yield batch_images, batch_images  # Autoencoder input = output

In [5]:
# Build the model
def build_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))  # Output layer (128x128x3)
    return model

In [6]:
# Compile the model
model = build_model()
model.summary()
model.compile(optimizer=Adam(), loss='mean_squared_error')

c:\Users\sn003\MyCodingEnv\Facial-reconstruction-from-CCTV-footage\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,731 (194.26 KB)

 Trainable params: 49,731 (194.26 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Set up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

# Train the model using the data generator
print("Starting training...")
# Recalculate the steps based on reduced batch size
steps_per_epoch = len(train_files) * 79 // BATCH_SIZE  # Adjusted steps per epoch
validation_steps = len(val_files) * 79 // BATCH_SIZE


Starting training...


In [8]:
# Re-run training with the modified generator
model.fit(data_generator(train_files, DATASET_PATH, BATCH_SIZE),
          steps_per_epoch=steps_per_epoch,
          epochs=EPOCHS,
          validation_data=data_generator(val_files, DATASET_PATH, BATCH_SIZE),
          validation_steps=validation_steps,
          callbacks=[early_stopping, model_checkpoint])

Epoch 1/10
   22/17330 ━━━━━━━━━━━━━━━━━━━━ 11:42:34 2s/step - loss: 0.1065

In [ ]:
# Function to display results
def display_results(original, reconstructed):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Reconstructed Image")
    plt.imshow(reconstructed)
    plt.axis('off')
    plt.show()

In [ ]:
# Test the model with a sample image from the validation set
sample_file = val_files[0]
data = np.load(os.path.join(DATASET_PATH, sample_file))
original_image = data['colorImages'][0]  # Take the first image for testing
original_image_resized = cv2.resize(original_image, IMAGE_SIZE) / 255.0
reconstructed_image = model.predict(np.expand_dims(original_image_resized, axis=0))[0]

# Display the original and reconstructed images
display_results(original_image_resized, reconstructed_image)

In [ ]:
# Save the model
model.save('facial_reconstruction_model.h5')
print("Model saved as 'facial_reconstruction_model.h5'.")